# Autocalibration interactive using autocalibrator.py

In [ ]:
%matplotlib inline

import sys
sys.path.append("../bin")

import glob
import os
from collections.abc import Callable, Sequence, Iterator, Mapping
from abc import ABC, abstractmethod
from typing import Any

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes

import numpy as np
import awkward as ak

from lgdo import lh5
from lgdo.lh5.exceptions import LH5DecodeError
from legendmeta import LegendMetadata
from dspeed.processors import get_multi_local_extrema

from autocalibrator import *

plt.rcParams["figure.figsize"] = (10, 4)

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta  = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250807T150028Z")
chmap_sipm = chmap.map("system", unique=False).spms
#requires recent legend-datasets
raw_keys = chmap_sipm.map("analysis.usability", unique=False).on.map("daq.rawid").keys()

In [ ]:
raw_dir = os.path.join(proj_dir, "data/tier/raw/phy/p15/r004_part")
dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p15r004dsp_part")
orig_dsp_dir = os.path.join(proj_dir, "data/tier/dsp/phy/p15/r004")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()
def gimme_raw_filename_from_dsp(dspfilename: str):
    return dspfilename.replace(dsp_dir, raw_dir).replace("tier_dsp", "tier_raw")
def gimme_orig_dsp_filename(dspfilename: str):
    # get the original dsp files so I get pulser info
    return dspfilename.replace(dsp_dir, orig_dsp_dir)

# Check uncalibrated spectra

In [ ]:
energies = get_energies(dsp_files, raw_keys, chmap, orig_dsp_file=[gimme_orig_dsp_filename(dsp) for dsp in dsp_files])

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.stairs(*gen_hist_by_range(get_energies(dsp_files, raw_keys, chmap)["S002"], (0,50)))
ax.stairs(*gen_hist_by_range(get_energies(list(map(gimme_orig_dsp_filename, dsp_files)), raw_keys, chmap)["S002"], (0,50)))

In [ ]:
#plot_all_pe_spectra(get_energies(gimme_orig_dsp_filename(dsp_files[0]), list(raw_keys), chmap))
#plot_all_pe_spectra(get_energies(list(map(gimme_orig_dsp_filename, dsp_files)), list(raw_keys), chmap))
_ = plot_all_pe_spectra(energies)

# Simple calibration

In [ ]:
# Default inputs
peakfinder_defaults = {
    "a_delta_min_in": 5e-3,
    "a_delta_max_in": 5e-3,
    "search_direction": 3,
    "a_abs_min_in": 1000,
    "a_abs_max_in": 1e-4,
    "min_peak_dist": 6,
    "peakdist_compare_margin": 1,
    "strict": True
}

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
simple_calibration(energies["S007"], {"quantile": 0.98, "nbins": 200}, peakfinder_defaults,
                   {}, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
simple_calibration(energies["S090"], {"quantile": 0.98, "nbins": 200}, peakfinder_defaults | {"double_peak": True}, {}, ax=ax)

In [ ]:
peakfinder_overrides={
    "S046": {"a_delta_min_in": 8e-3, "a_delta_max_in": 8e-3,},
    "S015": {"a_delta_min_in": 1e-2, "a_delta_max_in": 2e-2,},
    "S083": {"double_peak": True},
    "S090": {"double_peak": True},
    "S095": {"double_peak": True},
    "S096": {"double_peak": True, "a_delta_min_in": 1e-3, "a_delta_max_in": 1e-3,},
    "S098": {"double_peak": True},
}

In [ ]:
gen_hist_defaults = {"quantile": 0.98, "nbins": 200}

In [ ]:
simple_calib_output, _, _ = multi_simple_calibration(energies, gen_hist_defaults, peakfinder_defaults, {}, peakfinder_overrides=peakfinder_overrides, draw=True, nodraw_axes=True)

In [ ]:
calibrated_histos = get_calibrated_histograms(energies, simple_calib_output, (0, 5), 200)
_ = plot_all_pe_histograms(calibrated_histos)

In [ ]:
get_calibrated_PE_positions(simple_calib_output)

In [ ]:
advanced_calib_params_defaults = {
    "max_nr_gausspeaks": 3,
    "fit_range_prePE": 0.5,
    "fit_range_pastPE": 0.8,
    "model": "combo",
    "gauss_mean_range_low": 0.5,
    "gauss_mean_range_high": 0.5,
}

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
SiPM = "S095"
advanced_calibration(calibrated_histos[SiPM], get_calibrated_PE_positions(simple_calib_output)[SiPM], advanced_calib_params_defaults | {"fit_range_prePE": 0.5, "fit_range_pastPE": 0.6}, ax=ax, verbosity=100)

In [ ]:
advanced_calib_params_overrides = {
    "S040": {"fit_range_pastPE": 0.9},
    "S050": {"fit_range_pastPE": 1.1, "gauss_mean_range_high": 0.5, "model": "individual"},
    "S070": {"fit_range_prePE": 0.2, "fit_range_pastPE": 0.5},
    "S080": {"fit_range_prePE": 0.2, "fit_range_pastPE": 1.0, "gauss_mean_range_high": 1.5},
    "S083": {"fit_range_pastPE": 0.5},
    "S085": {"fit_range_prePE": 0.2},
    "S095": {"fit_range_pastPE": 0.6},
    "S096": {"fit_range_prePE": 0.2, "fit_range_pastPE": 0.4},
}

In [ ]:
advanced_calib_output, _, _ = multi_advanced_calibration(calibrated_histos, get_calibrated_PE_positions(simple_calib_output), advanced_calib_params_defaults, calibration_overrides=advanced_calib_params_overrides, draw=True)

In [ ]:
adv_calibrated_histos = get_calibrated_histograms(energies, combine_multiple_calibrations(simple_calib_output, advanced_calib_output), (0, 5), 200)
_ = plot_all_pe_histograms(adv_calibrated_histos, gridx=True)

### And now let's chain it all together!

In [ ]:
if False:
    full_calibration_chain(
        energies, 
        gen_hist_defaults=gen_hist_defaults, 
        peakfinder_defaults=peakfinder_defaults, 
        simple_calibration_defaults={}, 
        advanced_calibration_defaults=advanced_calib_params_defaults,
        peakfinder_overrides=peakfinder_overrides,
        advanced_calibration_overrides=advanced_calib_params_overrides,
        plot_interactive=True
    )

### Output config yaml for later reference

In [ ]:
store_config_file("../config/temp.yaml", 
    gen_hist_defaults=gen_hist_defaults, 
    peakfinder_defaults=peakfinder_defaults, 
    simple_calibration_defaults={}, 
    advanced_calibration_defaults=advanced_calib_params_defaults,
    peakfinder_overrides=peakfinder_overrides,
    advanced_calibration_overrides=advanced_calib_params_overrides)